In [5]:
from __future__ import division
from pymongo import MongoClient
from datetime import datetime
from collections import Counter
import pandas as pd
import json
from utils.data_manager import get_business_reviews, get_business

from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
client = MongoClient()
db = client.yelp

In [3]:
pipe = [{'$group': {'_id':'$business_id', 'count':{'$sum':1}}}, {'$sort': {'count':-1}}]
db.reviews.aggregate(pipe)['result']

[{u'_id': u'4bEjOyTaDG24SY5TxsaUNQ', u'count': 5558},
 {u'_id': u'lliksv-tglfUz1T3B3vgvA', u'count': 2911},
 {u'_id': u'CZjcFdvJhksq9dy58NVEzw', u'count': 2832},
 {u'_id': u'tFU2Js_nbIZOrnKfYJYBBg', u'count': 2781},
 {u'_id': u'6LM_Klmp3hOP0JmsMCKRqQ', u'count': 2551},
 {u'_id': u'eWPFXL1Bmu1ImtIa2Rqliw', u'count': 2459},
 {u'_id': u'DO3Gk17RyJVW7zYMCtYPnw', u'count': 2369},
 {u'_id': u'jf67Z1pnwElRSXllpQHiJg', u'count': 2345},
 {u'_id': u'34uJtlPnKicSaX1V8_tu1A', u'count': 2253},
 {u'_id': u'NGJDjdiDJHmN2xxU7KauuA', u'count': 2081},
 {u'_id': u'xyTJYlbE_MLouK6rCou6zg', u'count': 2004},
 {u'_id': u'-7yF42k0CcJhtPw51oaOqQ', u'count': 1970},
 {u'_id': u'FV16IeXJp2W6pnghTz2FAw', u'count': 1921},
 {u'_id': u'SsTxjxo8qvqBMvan1rzNzg', u'count': 1875},
 {u'_id': u'TWD8c5-P7w9v-2KX_GSNZQ', u'count': 1859},
 {u'_id': u'G0nUhYF2q4tnoOPwCjXLig', u'count': 1815},
 {u'_id': u'u9wjRhUjySkHPa_hG3kFOg', u'count': 1746},
 {u'_id': u'hW0Ne_HTHEAgGF1rAdmR-g', u'count': 1709},
 {u'_id': u'ripCiWZ0MblMZSLr

In [11]:
review_list = get_business_reviews('GzWiVvE6JDdBNowNVshItA')

In [12]:
text = review_list[0]['text']

In [13]:
print text

We set out to find Bavarian Point restaurant and stumbled into Zur-Kate by accident.  They are next door to each other, although I had never heard of Zur-Kate before.  What a stroke of luck.

The atmosphere was very cozy with lots of German flair.  The place was very crowded at 8pm, and everyone seemed to be having a very god time.  The first thing we noticed about the menu was the far lower prices than the ones on menus we had seen on line for other Valley German restaurants.  Many different kinds of wurst, schitzel and chops are available to choose from.  Each entree comes with a choice of German potato salad, spatzel, potato cake, home fries or french fries.  You also receive your choice of sauerkraut or red cabbage.  A salad bar is included, and basket of rye bread delivered to the table.  There is not an entree on the menu over $12, and you can add a wurst to any entree for $2.50.  Beers are in the $2-$4 range - where does that happen?  These prices are already amazing, but then y

In [47]:
# Find Negative Reviews
negative_reviews = []
for x in db.reviews.find({'stars':{'$lt': 3}}).sort('date').limit(100000):
    negative_reviews.append(x['text'].split())
print "finished neg"  
# Find Negative Reviews
positive_reviews = []
for x in db.reviews.find({'stars':{'$gt': 3}}).sort('date').limit(100000):
    positive_reviews.append(x['text'].split())

finished neg


In [49]:
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import movie_reviews
 
def word_feats(words):
    return dict([(word, True) for word in words])
 
negfeats = [(word_feats(x), 'neg') for x in negative_reviews]
posfeats = [(word_feats(x), 'pos') for x in positive_reviews]
 
negcutoff = int(len(negfeats)*3/4)
poscutoff = int(len(posfeats)*3/4)
 
trainfeats = negfeats[:negcutoff] + posfeats[:poscutoff]
testfeats = negfeats[negcutoff:] + posfeats[poscutoff:]
print 'train on %d instances, test on %d instances' % (len(trainfeats), len(testfeats))
 
classifier = NaiveBayesClassifier.train(trainfeats)
print 'accuracy:', nltk.classify.util.accuracy(classifier, testfeats)
classifier.show_most_informative_features()

train on 150000 instances, test on 50000 instances
accuracy: 0.84008
Most Informative Features
          unprofessional = True              neg : pos    =    150.4 : 1.0
                    2013 = True              neg : pos    =    107.0 : 1.0
                Horrible = True              neg : pos    =    102.1 : 1.0
                   rude! = True              neg : pos    =     92.3 : 1.0
                   Worst = True              neg : pos    =     86.5 : 1.0
                apology, = True              neg : pos    =     85.0 : 1.0
                   Worse = True              neg : pos    =     80.3 : 1.0
               horrible! = True              neg : pos    =     77.0 : 1.0
                    Rude = True              neg : pos    =     69.0 : 1.0
               Terrible. = True              neg : pos    =     66.3 : 1.0


In [64]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment import SentimentIntensityAnalyzer

In [65]:
sid = SentimentIntensityAnalyzer()
for review in positive_reviews[:10]:
    review = ' '.join(review)
    print review
    print sid.polarity_scores(review)
    print "-"*50

The best pizza on the planet. Seriously.
{'neg': 0.156, 'neu': 0.459, 'pos': 0.385, 'compound': 0.5423}
--------------------------------------------------
Hole in the wall burrito joint with the BEST beer-muchies burritos! Super combo with everything and sour creme.
{'neg': 0.0, 'neu': 0.637, 'pos': 0.363, 'compound': 0.8786}
--------------------------------------------------
The gold standard for casinos everywhere. Just bring a fat wallet, you won't find many $5 tables here.
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
--------------------------------------------------
Best pizza I've ever had. My favorite is the (extremely deep) deep dish sausage.
{'neg': 0.0, 'neu': 0.625, 'pos': 0.375, 'compound': 0.802}
--------------------------------------------------
Good stuff. Pricey by normal pizza standards.
{'neg': 0.0, 'neu': 0.674, 'pos': 0.326, 'compound': 0.4404}
--------------------------------------------------
Love their subs. Cheap and top shelf ingredients.
{'neg': 0.0, 